### deploy backend
+ generate token secret
  + encode values by (using -n to remove the generated line breaks) `echo -n "value" | base64`
  + copy the generated value to the corresponding secret.yaml file
+ generate configmap
`kubectl create configmap env-config -n farm --from-env-file=env_config.env`

+ test the Kind cluster svc from local computer. This forward the request from 127.0.0.1/:port to the service
`kubectl port-forward svc/fastapi-app 8000:8000 -n farm`
+ if your deployment server doesn't have a broswer, use another pod and curl
  + `kubectl run curl-client --rm -it --image=curlimages/curl -n farm -- /bin/sh` 
  + run `curl http://fastapi-app:8000` or `curl http://fastapi-app.farm.svc.cluster.local:8000`        

### deploy react fronend
+ pack react.js code to build production bundle
`npm run build`. This will generate code in dist\ folder
+ test the production bundle (make sure fastAPI is running locally on prot 8000) by
`npx http-server dist`
+ if everything works fine, build the docker
  + in .dockerignore file, inlcude
    + node_modules
    + dist
    + .env
  + to use K8s ingress with the Prefix path of web, add `base: '/webfront/',` in vite.config.js file. 
    + this will add the prefix 'webfront' to all the .js, .css files that embedded in index.html 
    + in Dockerfile, include `COPY --from=build /app/dist /usr/share/nginx/html/webfront`                 
  + include envrionment variable defined by `{$import.meta.env.}` in ARG of Dockerfile 
    + declare ARG VITE_API_URL that corresponds to`{$import.meta.env.VITE_API_URL}`
    + use `docker build --build-arg VITE_API_URL=http://127.0.0.1:8000 -t react-app:local .`
    + test locally by `docker run --rm -d -p 9001:80 react-app:local`
    + for K8s docker using `docker build --build-arg VITE_API_URL=http://app.local/fastapi -t react-app-demo .`
    + push to docker hub
    `docker tag react-app-demo huangyuan2000/react-app-demo:v1.0`
    `docker push huangyuan2000/react-app-demo:v1.0`

### install ingress
+ install ingress-nginx controller
  + ` helm repo add ingress-nginx https://kubernetes.github.io/ingress-nginx`                                     
  + `helm repo update`                                             
  + `helm install nginx-ingress ingress-nginx/ingress-nginx  --namespace ingress-nginx --create-namespace`
+ check the installation
  + `kubectl get ns` to find out ingress-nginx namespace
  + `kubectl get all -n ingress-nginx` to list the installed deployment and replicaset of controller 
+ install ingress resource for fastapi
  + make sure you have your cluster ip (or 127.0.0.1 if using your local machine for K8s cluster) with domain name app.local is in your /etc/hosts file (C:/Windows/System32/drivers/etc/hosts on windows)
  + deploy farm_ingress.yaml file



